In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_mypy

In [ ]:
import nbimporter
from AIBaseClass import ChessAI

# Aufgabe 02: Minimax (einfacher Materialwert)

Diese Klasse implementiert den Minimax-Algorithmus zur Berechnung des nächsten Zuges im Mittelspiel. Die Evaluierung eines Boards wird dabei durch Betrachtung des Materialwertes der Figuren realisiert.

Die Klasse wird um eine neue Konstante `DEPTH` erweitert. Diese gibt an, wie viele zukünftige Halbzüge bei der Berechnung des nächsten Zuges betrachtet werden sollen, d. h. welche Höhe der entsprechende Baum aller möglichen nächsten Halbzüge aufweist. Die Konstante wird als Parameter übergeben und im Konstruktor gesetzt.

In [ ]:
import chess

class Exercise02AI(ChessAI):
    """Chooses middle game moves using minimax algorithm and material values"""

    def __init__(self, player_name: str, search_depth: int = 2) -> None:
        super().__init__(player_name)
        self.DEPTH = search_depth

## Evaluierungsfunktion

Die Evaluierungsfunktion `evaluate` nimmt ein Board als Argument und berechnet eine Ganzzahl als Maß dafür, ob die gegebene Stellung für den weißen Spieler eher zu einem Sieg (positiver Wert) oder zu einer Niederlage (negativer Wert) führt.

Diese Implementierung verwendet dafür den Materialwert (Tauschwert in Bauerneinheiten) nach folgender Tabelle:

| Figurname  | Materialwert |
|---|---|
| Bauer (pawn) | 1  |
| Springer (knight) | 3  |
| Läufer (bishop) | 3  |
| Turm (rook) | 5  |
| Dame (queen) | 9  |

Die Berechnung wird in drei Schritten durchgeführt:

1. Es wird für jede Figur und für jede Farbe die Anzahl der Spielfiguren ermittelt.
2. Für jede Figur wird die Differenz der Anzahl pro Farbe gebildet ($\texttt{Weiß} - \texttt{Schwarz}$)
3. Jede Differenz wird mit dem figurenspezifischen Faktor multipliziert und zur Gesamtsumme addiert.

In [ ]:
class Exercise02AI(Exercise02AI): # type: ignore
    MATERIAL_VALUE_PAWN = 1
    MATERIAL_VALUE_KNIGHT = 3
    MATERIAL_VALUE_BISHOP = 3
    MATERIAL_VALUE_ROOK = 5
    MATERIAL_VALUE_QUEEN = 9

    def evaluate(self, board: chess.Board) -> int:
        """Evaluates a given board. 
        Returns a positive value if white has a better material value than black."""
        # Get the amount of pieces on the board per color 
        amount_pawn_white   = len(board.pieces(chess.PAWN,   chess.WHITE))
        amount_pawn_black   = len(board.pieces(chess.PAWN,   chess.BLACK))
        amount_knight_white = len(board.pieces(chess.KNIGHT, chess.WHITE))
        amount_knight_black = len(board.pieces(chess.KNIGHT, chess.BLACK))
        amount_bishop_white = len(board.pieces(chess.BISHOP, chess.WHITE))
        amount_bishop_black = len(board.pieces(chess.BISHOP, chess.BLACK))
        amount_rook_white   = len(board.pieces(chess.ROOK,   chess.WHITE))
        amount_rook_black   = len(board.pieces(chess.ROOK,   chess.BLACK))
        amount_queen_white  = len(board.pieces(chess.QUEEN,  chess.WHITE))
        amount_queen_black  = len(board.pieces(chess.QUEEN,  chess.BLACK))
        
        # Get difference
        diff_pawn   = amount_pawn_white   - amount_pawn_black
        diff_knight = amount_knight_white - amount_knight_black
        diff_bishop = amount_bishop_white - amount_bishop_black
        diff_rook   = amount_rook_white   - amount_rook_black
        diff_queen  = amount_queen_white  - amount_queen_black

        # Calculate material value
        return sum([
            self.MATERIAL_VALUE_PAWN * diff_pawn,
            self.MATERIAL_VALUE_KNIGHT * diff_knight,
            self.MATERIAL_VALUE_BISHOP * diff_bishop,
            self.MATERIAL_VALUE_ROOK * diff_rook,
            self.MATERIAL_VALUE_QUEEN * diff_queen
        ])

## Minimax

Die rekursive `minimax`-Funktion betrachtet bei einem gegebenen Board alle möglichen Stellungen (Boards) nach einer gegebenen Anzahl von Halbzügen (Tiefe) und gibt die beste Evaluierung (`evaluate`-Funktion) für den aktuellen Spieler zurück.

Die Konstante `LIMIT` wird als obere (+) bzw. untere (-) Schranke für den Vergleich innerhalb der ersten Iteration verwendet.

Folgende Fälle werden unterschieden:

1. Falls die Tiefe Null ist oder keine weitere Züge möglich sind, wird die Auswertung des aktuellen Boards zurückgegeben (Rekursionsende).
2. Falls Weiß am Zug ist wird jeder mögliche Zug rekursiv evaluiert und das Maximum der Ergebnisse zurückgegeben.
3. Falls Schwarz am Zug ist wird jeder mögliche Zug rekursiv evaluiert und das Minimum der Ergebnisse zurückgegeben.

In [ ]:
# Debugging start
from copy import deepcopy
# Debugging end
class Exercise02AI(Exercise02AI): # type: ignore
    LIMIT = 99999

    # Debugging start
    path_history: dict = {}
    # Debugging end

    def minimax(self, board: chess.Board, depth: int) -> int:
        """Searches the best value with given depth using minimax algorithm"""
        # Recursion abort case
        if depth == 0 or board.is_game_over():
            return self.evaluate(board)

        # White to play (positive numbers are good)
        if board.turn:
            maxEvaluation = -self.LIMIT
            for move in board.legal_moves:
                board.push(move)
                evaluation = self.minimax(board, depth - 1)
                # Debugging start
                if maxEvaluation < evaluation:
                    board_copy = deepcopy(board)
                    board_copy.pop()
                    Exercise02AI.path_history[str(board_copy) + str(depth)] = move
                # Debugging end
                board.pop()
                maxEvaluation = max(maxEvaluation, evaluation)
            assert maxEvaluation != -self.LIMIT, f"No evaluation was possible at {board.fen()} with depth {depth}!"  # Failsave
            return maxEvaluation

        # Black to play (negative numbers are good)
        else:
            minEvaluation = self.LIMIT
            for move in board.legal_moves:
                board.push(move)
                evaluation = self.minimax(board, depth - 1)
                # Debugging start
                if minEvaluation > evaluation:
                    board_copy = deepcopy(board)
                    board_copy.pop()
                    Exercise02AI.path_history[str(board_copy) + str(depth)] = move
                # Debugging end
                board.pop()
                minEvaluation = min(minEvaluation, evaluation)
            assert minEvaluation != self.LIMIT, f"No evaluation was possible at {board.fen()} with depth {depth}!"  # Failsave
            return minEvaluation

## Berechnung des besten Zuges

Die Funktion `get_next_middle_game_move` berechnet auf einem gegebenen Board den besten nächsten Zug. Dabei wird jeder mögliche Zug mithilfe der `minimax`-Funktion in der gewünschten Tiefe evaluiert und der Zug mit der besten Bewertung zurückgegeben.

In [ ]:
class Exercise02AI(Exercise02AI): # type: ignore
    def get_next_middle_game_move(self, board: chess.Board) -> chess.Move | None:
        """Gets the best next move"""
        if board.is_game_over():
            return None
        # Debugging start
        Exercise02AI.path_history = {}
        # Debugging end
        best_move_val = -self.LIMIT if board.turn else self.LIMIT
        final_move = None

        for move in board.legal_moves:
            board.push(move)
            move_val = self.minimax(board, self.DEPTH)
            # IF best value < current value AND white to play OR best value >= current value AND black to play
            if (best_move_val < move_val) != board.turn:
                best_move_val = move_val
                final_move = move
            board.pop()
        return final_move

## Debugging Bereich

Die folgenden Zellen enthalten Code zum Testen der oben implementierten Funktionen.

In [ ]:
# Debugging function
from IPython.display import display
def display_path(board_orig, move, depth):
    """Displays the move path to the best evaluation."""
    board = deepcopy(board_orig)
    display(board)
    board.push(move)
    display(board)
    for i in range(depth, 0, -1):
        move = Exercise02AI.path_history[str(board) + str(i)]
        board.push(move)
        display(board)

In [ ]:
# Test evaluate
board = chess.Board()
board.set_fen("7B/pbpk4/1p6/2n1pp2/6p1/4P3/P1P2PNP/RNq1K1R1 w - - 0 1")
player = Exercise02AI("Testplayer", 3)
player.evaluate(board)

In [ ]:
# Test get_next_middle_game_move
DEPTH = 3
board = chess.Board()
board.set_fen("4k3/8/2n5/7K/5q2/2N5/8/2B5 b - - 0 1")
player = Exercise02AI("Testplayer", DEPTH)
#print(player.minimax(board, DEPTH))
move = player.get_next_middle_game_move(board)
print(move)
display_path(board, move, DEPTH)